<h3>Step 1 : Import All Necessary Library</h3>

In [1]:
from pylab import *
from sklearn.metrics import precision_recall_fscore_support
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy
import numpy as np
import cv2
import glob
import csv
import math
import scipy.spatial.distance as dist

<h3>Step 2 : Define the Color Quantization Function</h3>

In [2]:
def colorQuantization(colorChannelImage, bins, max_color = 255):
    # change color variance to bins variance
    quant = np.array(colorChannelImage) * (bins / max_color)
    quant = np.floor(quant)
    quant[quant>=bins] = bins - 1
    
    return quant

In [3]:
def combineColorQuantization(image, B_bins, G_bins, R_bins):
    # extract matrix of color channel image
    B = image[:,:,0]
    G = image[:,:,1]
    R = image[:,:,2]
    
    # kuantisasikan tiap color channel
    B_quant = colorQuantization(B, B_bins)
    G_quant = colorQuantization(G, G_bins)
    R_quant = colorQuantization(R, R_bins)
    
    # combine the color quantization
    combine_quant = (B_bins * G_bins * R_quant) + (B_bins * G_quant) + B_quant
    
    return combine_quant

<h3>Step 3 : Define the Edge Quantization Function</h3>

In [4]:
def edgeQuantization(image, binsTheta) :
    # extract matrix of color channel image
    B = image[:,:,0]
    G = image[:,:,1]
    R = image[:,:,2]

    # sobel implementation
    Bx = cv2.Sobel(B,cv2.CV_64F,1,0,ksize=3)
    By = cv2.Sobel(B,cv2.CV_64F,0,1,ksize=3)
    Gx = cv2.Sobel(G,cv2.CV_64F,1,0,ksize=3)
    Gy = cv2.Sobel(G,cv2.CV_64F,0,1,ksize=3)
    Rx = cv2.Sobel(R,cv2.CV_64F,1,0,ksize=3)
    Ry = cv2.Sobel(R,cv2.CV_64F,0,1,ksize=3)

    # get |a| and |b|
    a = sqrt(Bx**2 + Gx**2 + Rx**2)
    b = sqrt(By**2 + Gy**2 + Ry**2)

    # get ab
    ab = (Bx*By) + (Gx*Gy) + (Rx*Ry)

    # image orientation
    (h, w) = a.shape
    #binsTheta = 18
    theta = np.zeros((h,w))

    # edge quantization
    for i in range (0, h) :
        for j in range (0, w) :
            if (a[i,j] == 0 or b[i,j] == 0):
                cosab1 = 0;
            else :
                cosab1 = ab[i,j]/(a[i,j]*b[i,j])
            theta1 = math.degrees(np.arccos(cosab1))
            if (math.isnan(theta1)):
                theta1 = 0
            theta[i,j] = math.floor(theta1 * (binsTheta/180))
            if (theta[i,j] >= binsTheta-1) :
                theta[i,j] = binsTheta-1
    
    return np.array(theta)

<h3>Step 4 : Define the Texton Search Function</h3>

In [5]:
def textonSearch(colorQuant, cBins, edgeQuant, eBins) :
    # define the shape of image
    (h, w) = colorQuant.shape

    color_img = np.zeros((h,w))
    edge_img = np.zeros((h,w))
    
    # sliding window check for all color channel
    for i in range (0, h, 2) :
        for j in range (0, w, 2) :
            
            # texton search for color
            cTemp = colorQuant[i:i+2,j:j+2]
            if (cTemp[0,0] == cTemp[0,1]) :    # texton type 1
                color_img[i,j] = colorQuant[i,j]
                color_img[i,j+1] = colorQuant[i,j+1]
            if (cTemp[0,0] == cTemp[1,0]) :    # texton type 2
                color_img[i,j] = colorQuant[i,j]
                color_img[i+1,j] = colorQuant[i+1,j]
            if (cTemp[0,0] == cTemp[1,1]) :    # texton type 3
                color_img[i,j] = colorQuant[i,j]
                color_img[i+1,j+1] = colorQuant[i+1,j+1]
            if (cTemp[1,0] == cTemp[1,1]) :    # texton type 4
                color_img[i+1,j] = colorQuant[i+1,j]
                color_img[i+1,j+1] = colorQuant[i+1,j+1]
            if (cTemp[0,1] == cTemp[1,1]) :    # texton type 5
                color_img[i,j+1] = colorQuant[i,j+1]
                color_img[i+1,j+1] = colorQuant[i+1,j+1]
            if (cTemp[0,1] == cTemp[1,0]) :    # texton type 6
                color_img[i,j+1] = colorQuant[i,j+1]
                color_img[i+1,j] = colorQuant[i+1,j]
                
            # texton search for edge
            eTemp = edgeQuant[i:i+2,j:j+2]
            if (eTemp[0,0] == eTemp[0,1]) :    # texton type 1
                edge_img[i,j] = edgeQuant[i,j]
                edge_img[i,j+1] = edgeQuant[i,j+1]
            if (eTemp[0,0] == eTemp[1,0]) :    # texton type 2
                edge_img[i,j] = edgeQuant[i,j]
                edge_img[i+1,j] = edgeQuant[i+1,j]
            if (eTemp[0,0] == eTemp[1,1]) :    # texton type 3
                edge_img[i,j] = edgeQuant[i,j]
                edge_img[i+1,j+1] = edgeQuant[i+1,j+1]
            if (eTemp[1,0] == eTemp[1,1]) :    # texton type 4
                edge_img[i+1,j] = edgeQuant[i+1,j]
                edge_img[i+1,j+1] = edgeQuant[i+1,j+1]
            if (eTemp[0,1] == eTemp[1,1]) :    # texton type 5
                edge_img[i,j+1] = edgeQuant[i,j+1]
                edge_img[i+1,j+1] = edgeQuant[i+1,j+1]
            if (eTemp[0,1] == eTemp[1,0]) :    # texton type 6
                edge_img[i,j+1] = edgeQuant[i,j+1]
                edge_img[i+1,j] = edgeQuant[i+1,j]
    
    # make color histogram
    cF = np.histogram(color_img.ravel(),cBins,[0,64])
    colorFeatures = (np.array(cF[0]) / 6) # perlu dibagi dg 6 meyesuaikan dg jumlah type texton yg digunakan
    
    # make edge histogram
    eF = np.histogram(edge_img.ravel(),eBins,[0,18])
    edgeFeatures = (np.array(eF[0]) / 6)
    
    # combine color and edge features
    features = []
    features.extend(colorFeatures)
    features.extend(edgeFeatures)
    
    return features

<h3>Step 5 : Define the GLCM Function</h3>

In [6]:
def GLCM(image) :
    # convert iamge to greyscale
    grey_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # make co-occurance matrix
    gm = greycomatrix(grey_img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, normed=True)
    (h,w) = gm[:,:,0,0].shape
    
    glcm_features = []
    
    # calculate energy, contrast, correlation, entropy
    # using scikit library
    energy = greycoprops(gm, 'energy')
    contrast = greycoprops(gm, 'contrast')
    correlation = greycoprops(gm, 'correlation')
    glcm_features.extend(energy.tolist())
    glcm_features.extend(contrast.tolist())
    glcm_features.extend(correlation.tolist())
    
    entropy = []
    for i in range (0, 4):
        e = np.abs(gm[:,:,0,i]*np.log2(gm[:,:,0,i]))
        e[isnan(e)] = 0
        entropy.append(sum(e))
        
    # using manual calculation
    """
    energy = []
    contrast = []
    correlation = []
    entropy = []
    # buat matriks perkalian antara baris & koom utk menghitung contrast
    mcross = np.zeros((h,w))
    mcross2 = np.zeros((h,w))
    mcross3 = np.zeros((h,w))
    for i in range (1, h+1):
        for j in range (1, w+1):
            mcross[i-1,j-1] = (i-j)**2
            mcross2[i-1,j-1] = (0-j)**2
            mcross3[i-1,j-1] = i*j
    # hitung energy, contrast, correalation, entropy
    for i in range (0, 4):
        energy.append(sum(gm[:,:,0,i]**2))
        if (i == 0):
            contrast.append(sum(mcross*gm[:,:,0,i]))
        else:
            contrast.append(sum(mcross2*gm[:,:,0,i]))
        correlation.append(sum(mcross3*gm[:,:,0,i]))
        e = np.abs(gm[:,:,0,i]*np.log2(gm[:,:,0,i]))
        e[isnan(e)] = 0
        entropy.append(sum(e))
    glcm_features.extend(energy)
    glcm_features.extend(contrast)
    glcm_features.extend(correlation)
    #glcm_features.extend(entropy)
    """
    return glcm_features, entropy

<h3>Step 6 : Define the Searcher Class</h3>

In [7]:
class Searcher:
    def __init__(self, train):
        # store the index path
        self.train = train
        
    def search(self, queryFeatures, limit=5):
        # initialize our dictionary of results
        results = {}
    
        # loop over the rows in the index
        for row in train:
            features = [float(x) for x in row[1:]]
                
            d = self.canberra_distance(features, queryFeatures)
            results[row[0]] = d
                
        results = sorted([(v,k) for (k,v) in results.items()])
        
        # return our (limited) results
        return results[:limit]
    
    def canberra_distance(self, histA, histB):
        #d = dist.canberra(histA, histB)
        
        # modified canberra
        M = len(histA)
        mA = mean(histA)
        mB = mean(histB)
        d = 0
        for i in range (0, M) :
            d = d + ( abs(histA[i] - histB[i]) / ((abs(histA[i] + mA) + abs(histB[i] + mB))))
        
        return d

<h3>Step 7 : The Indexing</h3>

In [8]:
# Describe path of data train and index file
datatrain_path = "dataset"
indexfile_path = "indexing/TextonIndex.csv"

# define the bins for quantization
colorBins = 64
R_Bins = 4
G_Bins = 4
B_Bins = 4
edgeBins = 18

# open the output index file for writing
output = open(indexfile_path, 'w')

# use glob to grab the image paths and logo over them
for imagePath in glob.glob(datatrain_path+'/*.jpg'):
    # extract the image ID (i.e. the unique filename) from the image path and load the image itself
    imageID = imagePath[imagePath.rfind('/') + 1:]
    image = cv2.imread(imagePath)
    
    # Color Quantization
    colorQuant = combineColorQuantization(image, B_Bins, G_Bins, R_Bins)

    # Edge Quantization
    edgeQuant = edgeQuantization(image, edgeBins)
    
    # Texton Search
    #features = []
    features = textonSearch(colorQuant, colorBins, edgeQuant, edgeBins)
    
    # GLCM
    glcm, en = GLCM(image)
    
    # write the features to file
    #features.extend(glcm)
    features.extend(glcm[0])
    features.extend(glcm[1])
    features.extend(glcm[2])
    features.extend(en)
    features = [str(f) for f in features]
    output.write('%s,%s\n' % (imageID, ','.join(features)))

print('Finish indexing. Alhamdulillah...')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in arccos
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\texture.py:109: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(image.dtype, np.float):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log2
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in multiply


Finish indexing. Alhamdulillah...


<h3>Step 8 : Evaluate Model Using Cross Validation</h3>

In [9]:
# Define path of testing data and indexing file
index_file = 'indexing/textonIndex.csv'

# define the bins for quantization
colorBins = 64
R_Bins = 4
G_Bins = 4
B_Bins = 4
edgeBins = 18

y_true = []
y_pred = []    

for i in range (1, 7) :
    train = []
    test = []
    
    with open(index_file) as f:
        reader = csv.reader(f)

        # loop over the rows in the index
        for row in reader:
            if (int(row[0].split('_')[1].split('.')[0]) == i):
                test.append(row)
            else :
                train.append(row)
       
        # close the reader
        f.close()
        
    for query in test :
        for i in range (0, 5):
            y_true.append(query[0].split('\\')[1].split('_')[0])
            
        searcher = Searcher(train)
        q = [float(i) for i in query[1:]]
        results = searcher.search(q)
            
        for (score, resultID) in results:
            # load the result image and store it to y_pred
            #image = imread(resultID)
            y_pred.append(resultID.split('\\')[1].split('_')[0])
    
# calculate the precision, recall, fscore
evaluate = precision_recall_fscore_support(y_true, y_pred, average='micro')
print (evaluate)

(0.5515789473684211, 0.5515789473684211, 0.5515789473684211, None)


<h2>Trial and Error</h2>

In [67]:
#image = cv2.imread('dataset/B1_1.jpg')
#grey_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
grey_img =[[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3]]
gm = greycomatrix(grey_img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=4, normed=True)
# buat matriks perkalian antara baris & koom utk menghitung contrast
h = 4
w = 4
mcross = np.zeros((h,w))
mcross2 = np.zeros((h,w))
mcross3 = np.zeros((h,w))
for i in range (1, h+1):
    for j in range (1, w+1):
        mcross[i-1,j-1] = (i-j)**2
        mcross2[i-1,j-1] = (0-j)**2
        mcross3[i-1,j-1] = i*j
# hitung energy, contrast, correalation, entropy
energy = []
contrast = []
correlation = []
entropy = []
for i in range (0, 4):
    energy.append(sum(gm[:,:,0,i]**2))
    if (i == 0):
        contrast.append(sum(mcross*gm[:,:,0,i]))
    else:
        contrast.append(sum(mcross2*gm[:,:,0,i]))
    correlation.append(sum(mcross3*gm[:,:,0,i]))
    e = np.abs(gm[:,:,0,i]*np.log2(gm[:,:,0,i]))
    e[isnan(e)] = 0
    entropy.append(sum(e))
print (energy)
print (contrast)
print (correlation)
print (entropy)

[0.3333333333333333, 0.3333333333333333, 0.25, 0.3333333333333333]
[1.0, 9.666666666666666, 7.5, 4.666666666666667]
[6.666666666666667, 6.666666666666667, 7.5, 6.666666666666667]
[1.584962500721156, 1.584962500721156, 2.0, 1.584962500721156]


C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\texture.py:109: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(image.dtype, np.float):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log2
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply
